### Append Path

In [2]:
import sys

sys.path.append("..")

%reload_ext autoreload
%autoreload 2

### Load Data

In [3]:
import pandas as pd

df = pd.read_csv("../data/processed/feature_engineered_data.csv")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18296\3085591209.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/feature_engineered_data.csv")


### Train Test Split

In [5]:
from src.modeling import split_data

X_train, X_test, y_train, y_test = split_data(
    df, target_col="SolarGeneration", test_size=0.2
)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (568003, 27)
X_test shape: (142001, 27)


### Model Training

In [8]:
from src.modeling import train_model

model = train_model(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Params: {'num_leaves': 50, 'n_estimators': 1000, 'max_depth': -1, 'learning_rate': 0.01}

Training Complete.


### Model Evaluation

In [9]:
from src.modeling import evaluate_model

preds = evaluate_model(model, X_test, y_test)

MAE (Mean Absolute Error): 2.6007 kWh
RMSE (Root Mean Sq Error): 5.1724 kWh
R2 Score: 0.7927


### Visualization

In [4]:
import matplotlib.pyplot as plt

results = pd.DataFrame({"Actual": y_test, "Predicted": preds})

plt.figure(figsize=(15, 6))
plt.plot(results["Actual"].values[:100], label="Actual", color="blue", alpha=0.5)
plt.plot(
    results["Predicted"].values[:100], label="Predicted", color="red", linestyle="--"
)
plt.title("Solar Generation Forecast: 1 Day Ahead")
plt.xlabel("Time 15-Minute Intervals")
plt.ylabel("Generation (kWh)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

NameError: name 'y_test' is not defined

### Feature Importance

In [1]:
from lightgbm import plot_importance

fig, ax = plt.subplots(figsize=(10, 8))
plot_importance(model, ax=ax, max_num_features=10)
plt.show()

NameError: name 'plt' is not defined

### Save Model

In [8]:
from src.modeling import save_model

save_model(model, "../models/solar_lightgbm.pkl")

Model saved to ../models/solar_lightgbm.pkl
